In [1]:
import pandas as pd

In [2]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [3]:
from nltk.corpus import stopwords
stop=[]
stop.extend(stopwords.words('french'))
stop.extend(stopwords.words('spanish'))
stop.extend(stopwords.words('english'))


In [286]:
len(stop)

647

In [4]:
moresa = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/more_data_filterd.csv')

In [6]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stop:
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [7]:
df_train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_test = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/test.csv') 
print df_test.shape
df_train.shape

(18543, 8)


(43266, 9)

In [8]:
df_train.isnull().sum()

Complaint-ID                      0
Date-received                     0
Transaction-Type                  0
Complaint-reason                  0
Company-response              22506
Date-sent-to-company              0
Complaint-Status                  0
Consumer-disputes              7698
Consumer-complaint-summary        0
dtype: int64

In [9]:
df_train['Complaint-reason'] = df_train['Complaint-reason'].apply(preprocess_data)
print 'train ---done'
df_test['Complaint-reason'] = df_test['Complaint-reason'].apply(preprocess_data)

train ---done


In [10]:
df_train['Transaction-Type'] = df_train['Transaction-Type'].apply(preprocess_data)#.map(lambda x : x.replace(' ','_'))
df_test['Transaction-Type'] = df_test['Transaction-Type'].apply(preprocess_data)#.map(lambda x : x.replace(' ','_'))


In [11]:
df_train['Consumer-complaint-summary'] = df_train['Consumer-complaint-summary'].apply(preprocess_data)
print 'train-done'
df_test['Consumer-complaint-summary'] = df_test['Consumer-complaint-summary'].apply(preprocess_data)

train-done


In [13]:
df_train['Consumer-complaint-summary'] = df_train['Consumer-complaint-summary'].map(lambda x : re.sub("x+","",x))
df_test['Consumer-complaint-summary'] = df_test['Consumer-complaint-summary'].map(lambda x : re.sub("x+","",x))

In [19]:
# df_train['Company-response'] = df_train['Company-response'].apply(preprocess_data)
# print 'train-done'

# df_test['Consumer-disputes'].fillna(df_test['Consumer-disputes'].mode()[0], inplace=True)
# df_test['Company-response'].fillna(df_test['Company-response'].mode()[0], inplace=True)

df_imp = df_train[ ['Transaction-Type','Consumer-complaint-summary','Complaint-reason','Complaint-Status']]

In [20]:
# df_test['Company-response'] = df_test['Company-response'].apply(preprocess_data)
df =  pd.concat([df_imp,moresa],axis=0)

In [21]:
df.head()

,Transaction-Type,Consumer-complaint-summary,Complaint-reason,Complaint-Status
0,mortgage,seterus inc po fau rapport aupr principau bure...,loan servicing payment escrow account,Closed with explanation
1,credit reporting,requ faillite chapitre po hawaii incluait...,incorrect information credit report,Closed with non-monetary relief
2,bank account service,preparando vuelo regreso vea funeral famil...,using debit atm card,Closed with explanation
3,debt collection,loan paid year moved va moved month pay...,contd attempt collect debt owed,Closed with explanation
4,credit card,jai obtenu compte cr dit soins imm diatement ...,payoff process,Closed with explanation


In [22]:
df.shape

(110072, 4)

In [35]:
df.to_csv('train_pre_with_more.csv',index=None)
df_test.to_csv('test_pre.csv',index=None)

In [26]:
df_t = df_test

In [48]:
# cd = pd.get_dummies(df['Consumer-disputes'])
# cd_t = pd.get_dummies(df_t['Consumer-disputes'])

In [136]:
# df = pd.concat([cd,df],axis=1)
# df_t = pd.concat([cd_t,df_t],axis=1)

In [28]:
df_t.head()

,Complaint-ID,Date-received,Transaction-Type,Complaint-reason,Company-response,Date-sent-to-company,Consumer-disputes,Consumer-complaint-summary
0,Te-1,8/18/2016,bank account service,account opening closing management,Company has responded to the consumer and the ...,8/18/2016,No,called citibank open checking account promot...
1,Te-2,4/18/2016,debt collection,communication tactic,Company believes it acted appropriately as aut...,4/20/2016,No,im struggling financially called offered pay ...
2,Te-3,3/23/2016,credit reporting,incorrect information credit report,NaN,3/23/2016,No,automatic payment configured capital line pe...
3,Te-4,6/26/2017,student loan,dealing lender servicer,NaN,6/26/2017,NaN,submitted request current loan provider stude...
4,Te-5,5/13/2016,credit reporting,incorrect information credit report,Company has responded to the consumer and the ...,5/13/2016,No,state ta lien filed however never notified ...


In [45]:
df["merge"] =  df["Consumer-complaint-summary"]+ " " +df["Complaint-reason"]+ " " +df['Transaction-Type']# #+ " "+\
# + " " + df['Company-response']


In [46]:
df_t["merge"] =  df_t["Consumer-complaint-summary"]+ " " +df_t["Complaint-reason"]+" "+df_t['Transaction-Type']#+df_t["Complaint-reason"] #+ " "+\
# + " " + df_t['Company-response']


In [47]:
all_text  = []
all_text = list(df['merge'].values.astype('U')) + list(df_t['merge'].values.astype('U'))


In [48]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(sublinear_tf=True,use_idf=True, min_df=10, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
tfidf.fit(all_text)


TfidfVectorizer(analyzer=u'word', binary=False, decode_error=u'strict',
        dtype=<type 'numpy.float64'>, encoding='latin-1', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=10,
        ngram_range=(1, 2), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=True,
        token_pattern=u'(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [49]:
features = tfidf.transform(df['merge'].values.astype('U'))
labels = df['Complaint-Status']

features.shape

(110072, 141601)

In [50]:
features_t = (tfidf.transform(df_t['merge'].values.astype('U')))
features_t.shape

(18543, 141601)

In [303]:
len(labels)
print 'paucha'

paucha


In [51]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.naive_bayes import MultinomialNB

from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
models = [
    LinearSVC()

]
CV = 5
cv_df = pd.DataFrame(index=range(CV * len(models)))
entries = []
for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, labels, scoring='f1_weighted', cv=CV)
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))
        print (model_name, fold_idx, accuracy)
cv_df = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])

('LinearSVC', 0, 0.7532762342222195)
('LinearSVC', 1, 0.7493368944125787)
('LinearSVC', 2, 0.7448342818212551)
('LinearSVC', 3, 0.7597540568797019)
('LinearSVC', 4, 0.760401425507926)


In [52]:
clf = OneVsRestClassifier(LinearSVC()).fit(features,labels)

In [53]:
id_ = list(df_test['Complaint-ID'])

In [54]:
y_pred = clf.predict(features_t)

In [55]:
result = pd.DataFrame(id_,columns=['Complaint-ID'])
result['Complaint-Status'] = (y_pred)

In [56]:
result.to_csv('/home/prashant/Downloads/c3cc8568-0-dataset/svm_consumer_moresa1.csv',index=None)

In [127]:
from langdetect import detect

In [130]:
list(df_t['Consumer-complaint-summary'])

[u'  called citibank open checking account promotional code mile   spoke  part promotional fine comply certain condition epense payment invoice met provide evidence proof promotion code applied correctly account sent secure message citi confirm    replied could see account request determine promotion code applied would eligible promotion therefore continued comply established requirement paying monthly fee checking account based response mile credited account got touch citi     confirmed promotion code attached account intended citi unable provide information  went ahead fulfilled part deal provide information contradicted previous response',
 u'im struggling financially called offered pay  dollar month debt could get back refused accept payment said would garnishing salary even said would sleep well night knowing would able eat got salary',
 u' automatic payment configured capital line  personal account  amount incorrectly configured first statement reflected partial month payment mad

In [133]:
cnt=0
for i in list(df_t['Consumer-complaint-summary']):
    
    if detect(i) != 'en':
        cnt +=1

In [134]:
cnt

6636